# 划分 Wikipedia 学科, 使用 core 学科

In [1]:
import pymongo
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
from dotenv import dotenv_values
import tqdm
import collections

env_config = dotenv_values("../.env")

In [2]:
database_name = f"wikipedia_dump_v{env_config['WIKI_VERSION']}"
wikipedia_collection = pymongo.MongoClient(env_config['Mongo_Url'])[database_name]["revision_complete"]

In [ ]:
# 找到所有存在 page linksin 的 category

# category_set = set()

# for doc in tqdm.tqdm(wikipedia_collection.find({'page_category_links_out':{'$ne':None}},{'page_category_links_out':1})):
#     category_set.update(doc['page_category_links_out'])

# len(category_set)

In [20]:
# 找到所有存在 page linksin 的 category

# category_set = set(wikipedia_collection.distinct('page_category_links_out'))
# len(category_set)

In [3]:
# 找到匹配的条件的 category core name
subject_list = [ "Mathematics", "Physics", "Computer science", "Engineering disciplines", "Medicine",
		"Biology", "Chemistry", "Materials science", "Geology", "Geography", "Environmental science",
		"Economics", "Sociology", "Psychology", "Political science", "Philosophy", "Business", "Art",
		"History" ]

core_key_1 = ["Subfields of", "Areas of" "Fields of", "Branches of", "Subdivisions of"]

core_key_2 = ["by field", "by fields", "of field", "by specialty"]

subject_key_map = collections.defaultdict(set)
total_map = set()

for name in subject_list:
    subject_key_map[name] = set()
    low_name = name.lower()
    for key in core_key_2:
        subject_key_map[name].add(f"{low_name} {key.lower()}")
        total_map.add(f"{low_name} {key.lower()}")
    for key in core_key_1:
        subject_key_map[name].add(f"{key.lower()} {low_name}")
        total_map.add(f"{key.lower()} {low_name}")
    
    total_map.add(low_name)
        
# subject_key_map

In [ ]:
# # 生成子学科到主学科的映射, 子学科区分大小写

# # core cat
# subject_core_cat_map = collections.defaultdict(set)

# # 包含主学科的 cat set
# subject_chief_cat_map = collections.defaultdict(set)

# for cat_name in tqdm.tqdm(category_set):
#     cat_name_lower = cat_name.lower()
#     if cat_name_lower in total_map:
#         for subject_name , cat_set in subject_key_map.items():
#             if cat_name_lower in cat_set:
#                 subject_core_cat_map[subject_name].add(cat_name)
#             elif cat_name_lower == subject_name.lower():
#                 subject_chief_cat_map[subject_name].add(cat_name)
     
# reverse_subject_core_cat_map = {}
# for name, items in subject_core_cat_map.items():
#     for sub_name in items:
#         reverse_subject_core_cat_map[sub_name] = name
        
# reverse_subject_chief_cat_map = {}
# for name, items in subject_chief_cat_map.items():
#     for sub_name in items:
#         reverse_subject_chief_cat_map[sub_name] = name

100%|██████████| 2526905/2526905 [00:01<00:00, 1454956.36it/s]


In [ ]:
# # reverse_subject_chief_cat_map

# # subject_core_cat_map
# # subject_chief_cat_map

# for name in subject_list:
#     if name not in subject_core_cat_map:
#         print(subject_chief_cat_map[name])

{'engineering disciplines', 'Engineering disciplines'}
{'Medicine', 'medicine'}
{'Materials science', 'Materials Science', 'materials science'}
{'Environmental science', 'Environmental Science'}
{'Business', 'business'}
{'Art'}


# 逻辑处理
要计算的学科子图

1. 一层 core subject category page (有多个都用一层), 无 core subject category 的就用一层 subject category page
2. 扩展到二层, 三层, 每年独立计算 category 层级

In [5]:
def get_category_range(year:int):
    
    # 找到当前所有的 category linksin
    category_set = set()
    for doc in wikipedia_collection.find({'ns':0,'year_tags':year, 'page_category_links_out':{'$ne':None}},{'page_category_links_out':1}):
        category_set.update(doc['page_category_links_out'])
    print(year, len(category_set))
    
    # 生成子学科到主学科的映射, 子学科区分大小写

    # core cat
    subject_core_cat_map = collections.defaultdict(set)

    # 包含主学科的 cat set
    subject_chief_cat_map = collections.defaultdict(set)

    for cat_name in tqdm.tqdm(category_set):
        cat_name_lower = cat_name.lower()
        if cat_name_lower in total_map:
            for subject_name , cat_set in subject_key_map.items():
                if cat_name_lower in cat_set:
                    subject_core_cat_map[subject_name].add(cat_name)
                elif cat_name_lower == subject_name.lower():
                    subject_chief_cat_map[subject_name].add(cat_name)
                    
    # 对没有 core cat 的学科进行补充
    for name in subject_list:
        if name not in subject_core_cat_map:
            for item in subject_chief_cat_map[name]:
                subject_core_cat_map[name].add(item)
    
    # 生成反向 map
    reverse_subject_core_cat_map = {}
    for name, items in subject_core_cat_map.items():
        for sub_name in items:
            reverse_subject_core_cat_map[sub_name] = name
            
    # 获取了当年的所有主学科和子学科的映射
    lv1_subject_dict = collections.defaultdict(set)
    lv2_subject_dict = collections.defaultdict(set)
    lv3_subject_dict = collections.defaultdict(set)
    
    # lv2
    for cat_name, subject_name in reverse_subject_core_cat_map.items():
        for doc in wikipedia_collection.find({'ns':14,'year_tags':year, 'page_category_links_out': cat_name}):
            sub_cat_name = doc['title'].split(":")[-1]
            lv2_subject_dict[subject_name].add(sub_cat_name)
            
    # lv3
    for subject_name, cat_name_list in lv2_subject_dict.items():
        for cat_name in cat_name_list:
            # lv3 包含lv2
            lv3_subject_dict[subject_name].add(cat_name)
            for doc in wikipedia_collection.find({'ns':14,'year_tags':year, 'page_category_links_out': cat_name}):
                sub_cat_name = doc['title'].split(":")[-1]
                lv3_subject_dict[subject_name].add(sub_cat_name)
    
    # lv1 
    for cat_name, subject_name in reverse_subject_core_cat_map.items():
        lv1_subject_dict[subject_name].add(cat_name)
        # lv2, lv3 包含 lv1
        lv2_subject_dict[subject_name].add(cat_name)
        lv3_subject_dict[subject_name].add(cat_name)
    
    return lv1_subject_dict, lv2_subject_dict, lv3_subject_dict


wikipedia_collection.create_index([('core_subject_tag', pymongo.ASCENDING)],background=True,sparse=True)

def calculate_yearly_subject_entropy(year:int):
    
    lv1_subject_dict, lv2_subject_dict, lv3_subject_dict = get_category_range(year)
    
    core_tag_map = collections.defaultdict(set)
    
    for subject_name, cat_name_list in lv1_subject_dict.items():
        for cat_name in cat_name_list:
            for doc in wikipedia_collection.find({'ns':0, 'page_category_links_out':cat_name, 'year_tags':year},{'_id':1}):
                core_tag_map[doc['_id']].add(f"lv1-{subject_name}-{year}")
                
    for subject_name, cat_name_list in lv2_subject_dict.items():
        for cat_name in cat_name_list:
            for doc in wikipedia_collection.find({'ns':0, 'page_category_links_out':cat_name, 'year_tags':year},{'_id':1}):
                core_tag_map[doc['_id']].add(f"lv2-{subject_name}-{year}")
                
    for subject_name, cat_name_list in lv3_subject_dict.items():
        for cat_name in cat_name_list:
            for doc in wikipedia_collection.find({'ns':0, 'page_category_links_out':cat_name, 'year_tags':year},{'_id':1}):
                core_tag_map[doc['_id']].add(f"lv3-{subject_name}-{year}")
          
    bulk_operate_list = []      
    for doc_id, tag_set in core_tag_map.items():
        bulk_operate_list.append(
            pymongo.UpdateOne(
                {'_id':doc_id},
                {'$addToSet':{'core_subject_tag':{"$each":list(tag_set)}}}
            )
        )
    wikipedia_collection.bulk_write(bulk_operate_list)
    print('bulk_operate_list', len(bulk_operate_list))
                
lv1_subject_dict, lv2_subject_dict, lv3_subject_dict = get_category_range(2007)

2007 262895


100%|██████████| 262895/262895 [00:00<00:00, 1433113.06it/s]


In [12]:
list(lv3_subject_dict.keys())

['Psychology',
 'Sociology',
 'Geography',
 'Philosophy',
 'Political science',
 'Mathematics',
 'Physics',
 'Computer science',
 'Engineering disciplines',
 'Medicine',
 'Biology',
 'Chemistry',
 'Materials science',
 'Geology',
 'Environmental science',
 'Economics',
 'Business',
 'History']

In [13]:
import time
time.sleep(600)

# 测试成功
for year in range(2003,2024+1):
    calculate_yearly_subject_entropy(year)

2003 22


100%|██████████| 22/22 [00:00<00:00, 100626.70it/s]

bulk_operate_list 1


2004 36037


100%|██████████| 36037/36037 [00:00<00:00, 1381413.61it/s]


...

bulk_operate_list 152393
2023 1638406


100%|██████████| 1638406/1638406 [00:01<00:00, 1463403.45it/s]


bulk_operate_list 144258
2024 1715068


100%|██████████| 1715068/1715068 [00:01<00:00, 1479130.31it/s]


bulk_operate_list 148805
